# Data Mining Homework 4

## Problem 4

In [1]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
## Where 1 = true, 0 = false 
data = {'outlook': ['sunny','sunny','overcast','rainy','rainy','rainy','overcast','sunny','sunny','rainy','sunny','overcast','overcast','rainy'],
        'temp':[85,80,83,70,68,65,64,72,69,75,75,72,81,71],
        'humidity': [85,90,86,96,80,70,65,95,70,80,70,90,75,91],
        'windy': ['false','true','false','false','false','true','true','false','false','false','true','true','false','true'],
        'play' : ['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']
        }
df= pd.DataFrame (data, columns = ['outlook','temp','humidity','windy','play'])
df.head()

,outlook,temp,humidity,windy,play
0,sunny,85,85,false,no
1,sunny,80,90,true,no
2,overcast,83,86,false,yes
3,rainy,70,96,false,yes
4,rainy,68,80,false,yes


In [2]:
df['temp>=75'] = np.where(df['temp'] >= 75, 'true', 'false')
df['humidity>=80'] = np.where(df['humidity']>= 80,'true','false')
df

,outlook,temp,humidity,windy,play,temp>=75,humidity>=80
0,sunny,85,85,false,no,true,true
1,sunny,80,90,true,no,true,true
2,overcast,83,86,false,yes,true,true
3,rainy,70,96,false,yes,false,true
4,rainy,68,80,false,yes,false,true
5,rainy,65,70,true,no,false,false
6,overcast,64,65,true,yes,false,false
7,sunny,72,95,false,no,false,true
8,sunny,69,70,false,yes,false,false
9,rainy,75,80,false,yes,true,true


In [3]:
df = df.drop(columns = ['temp','humidity'])

In [11]:
df['Play'] = df['play']
df = df.drop(columns = ['play'])
df

,outlook,windy,temp>=75,humidity>=80,Play
0,sunny,false,true,true,no
1,sunny,true,true,true,no
2,overcast,false,true,true,yes
3,rainy,false,false,true,yes
4,rainy,false,false,true,yes
5,rainy,true,false,false,no
6,overcast,true,false,false,yes
7,sunny,false,false,true,no
8,sunny,false,false,false,yes
9,rainy,false,true,true,yes


 ## Description
 'temp': [85,80,83,70,68,65,64,72,69,75,75,72,81,71],
        'humidity': [85,90,86,96,80,70,65,95,70,80,70,90,75,91],
        
So I know this is probably not the best but when I wrote it with the actual number values Temp was the max info gain which gave us difficult results as both temp and humidity are continous variables. So instead, I chose values of 75 and 80 respectively to determine a true or false value. The results may be different, but it allowed me to predict better

Part 1 Using Entropy/Information Gain

In [36]:
def find_entropy(df):
    Class = df.keys()[-1]
    entropy = 0
    values = df.Play.unique()
    for value in values:
        fraction = df.Play.value_counts()[value]/len(df.Play)
        entropy += -fraction*np.log2(fraction)
    return entropy

In [37]:
totEntropy = find_entropy(df)

In [38]:
def find_entropy_key(df,attribute):
    Class = df.keys()[-1]
    target_variables = df.Play.unique()
    variables = df[attribute].unique()
    entropy_attribute1 = 0
    for variable in variables:
        entropy_individual = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.Play == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy_individual += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy_attribute1 += -fraction2*entropy_individual
    return abs(entropy_attribute1)

In [39]:
tempEntropy = find_entropy_key(df,'temp>=75')
outlookEntropy = find_entropy_key(df,'outlook')
windyEntropy = find_entropy_key(df,'windy')
humidityEntropy = find_entropy_key(df,'humidity>=80')

In [40]:
## Info Gain
tempGain = totEntropy - tempEntropy
outlookGain = totEntropy - outlookEntropy
windyGain = totEntropy - windyEntropy
humidityGain = totEntropy - humidityEntropy
print([tempGain, outlookGain, windyGain, humidityGain])

[0.0013397424044421236, 0.24674981977443977, 0.048127030408270155, 0.04533417202914514]


In [41]:
# Find the most valu
def maxInfoGain(df):
    Entropy_val = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_key(df,key))
    return df.keys()[:-1][np.argmax(IG)]   

In [42]:
maxInfoGain(df)

'outlook'

In [43]:
def create_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop = True)

In [44]:
def buildTree(df, tree = None):
    Class = df.keys()[-1]
    # Allow for the comparison of methods
    
    node = maxInfoGain(df)
    attValue = np.unique(df[node])
    if tree is None:                    
        tree={}
        tree[node] = {}
    for value in attValue:
        
        subtable = create_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Play'],return_counts=True)                        
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 
                   
    return tree

In [45]:
tree = buildTree(df)

In [46]:
import pprint
pprint.pprint(tree)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'false': 'yes', 'true': 'no'}},
             'sunny': {'humidity>=80': {'false': 'yes', 'true': 'no'}}}}


In [47]:
def predict(inst,tree):
    for nodes in tree.keys():
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;
    return prediction

In [48]:
dataDict = {'outlook': 'overcast','temp>=75' : 'false','humidity>=80' :'false','windy': 'false'}
# Temp is 60, Humidity is 62 --> Both are false 
inst = pd.Series(dataDict)

In [49]:
predict(inst,tree)

'yes'

## Now Change Methods (Info Gain vs Gini Index vs Gain Ratio)

In [87]:
import collections


In [89]:
## Since Gain Ratio is Split() / Gain 
def splitInfo(df,attribute):
    tot = len(df)
    counterDic = dict(collections.Counter(df[attribute]))
    totalValue = []
    for key in counterDic.keys():
        fraction = (counterDic[key] / tot)
        tempVal = - (fraction*np.log2(fraction))
        totalValue.append(tempVal)
    return sum(totalValue)
def gainRatio(df,attribute):
    if attribute == 'outlook':
        split = splitInfo(df,attribute)
        return   outlookGain / split
    if attribute == 'windy':
        split = splitInfo(df,attribute)
        return  windyGain / split
    if attribute == 'temp>=75':
        split = splitInfo(df,attribute)
        return   tempGain / split
    if attribute == 'humidity>=80':
        split = splitInfo(df,attribute)
        return   humidityGain / split
    
def maxGainRatio(df):
    colNames = []
    for col in df.columns:
        colNames.append(col)
    colNames = colNames[:-1]
    ratioPerAt = {}
    for i in colNames: 
        ratioPerAt.update({i : gainRatio(df,i)})
    keyMax = max(ratioPerAt, key = ratioPerAt.get)
    return keyMax


In [119]:
## Gini Index
def GiniIndex(df):
    tot = len(df)
    colNames = []
    for col in df.columns:
        colNames.append(col)
    colNames = colNames[:-1]
    giniDic = {}
    for i in colNames:
        tempDic = attributeDisection(df,i)
        tempValues = []
        for key in tempDic.keys():
            tempValues.append(tempDic[key] / tot)
        coef = len(tempValues)
        val = 1
        for x in tempValues:
            val = val * x
        gini = coef*val
        giniDic.update({i : gini})
    return giniDic
    
def attributeDisection(df,attribute):
    return dict(collections.Counter(df[attribute]))
def minGini(df):
    tempDic = GiniIndex(df)
    keyMin = min(tempDic, key =tempDic.get)
    return keyMin


# Applying to Tree Construction

Now that we have established methods to determine root nodes. We will modify our tree from above to compare results.

In [120]:
def buildTree2(df,attributeCriteria, tree = None):
    Class = df.keys()[-1]
    # Allow for the comparison of methods
    if attributeCriteria == 'Gini':
        node = minGini(df)
    if attributeCriteria == 'Gain':
        node = maxInfoGain(df)
    if attributeCriteria == 'GainRatio':
        node = maxGainRatio
    attValue = np.unique(df[node])
    if tree is None:                    
        tree={}
        tree[node] = {}
    for value in attValue:
        
        subtable = create_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Play'],return_counts=True)                        
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 
                   
    return tree

In [122]:
treeGini = buildTree2(df,'Gini')
treeGain = buildTree2(df,'Gain')
treeGainRatio = buildTree2(df,'GainRatio')

In [123]:
#Gini
pprint.pprint(treeGini)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'false': 'yes', 'true': 'no'}},
             'sunny': {'humidity>=80': {'false': 'yes', 'true': 'no'}}}}


In [124]:
#Gain
pprint.pprint(treeGain)

{'outlook': {'overcast': 'yes',
             'rainy': {'windy': {'false': 'yes', 'true': 'no'}},
             'sunny': {'humidity>=80': {'false': 'yes', 'true': 'no'}}}}


In [125]:
#GainRatio
pprint.pprint(treeGainRatio)

{<function maxGainRatio at 0x7fe1e81e2d30>: {'overcast': 'yes',
                                             'rainy': {'windy': {'false': 'yes',
                                                                 'true': 'no'}},
                                             'sunny': {'humidity>=80': {'false': 'yes',
                                                                        'true': 'no'}}}}


## Results

When we implemented the attribute criteria selection, we recieved the same decision tree every time. 